Rosetta Design
===

In this exercise we'll be designing some core residues of the Lambda Repressor-Operator complex to examine whether alternate packing arrangements are possible. We will start out with some fixed-backbone design on seven core residues, then move onto flexible-backbone and finally flexible-backbone with layered design. Then, in teams, you will generate a few models from each of these protocols, and compare your sequences to the results of an experimental mutagenesis selection from Lim, et al (1989) (https://www.nature.com/articles/339031a0.pdf).


### Setup

Before we do anything, update RosEasy:

```bash
cd /path/to/class/folder/
cd roseasy
git pull
cd ..
```

Now activate your `protiendesign` environemnt and clone a GitHub repository containing our input files.

```bash
conda activate proteindesign
git clone https://github.com/Kortemme-Lab/protein_design_course_files.git
```


Next we will set up a RosEasy workspace to make file management during the design process easier. RosEasy will ask you for a few input files. Not all are required for every design task, so for those you can just hid \<enter\>. We will need an input PDB file and a resfile. A resfile is a file format that Rosetta can use to determine which amino acids are allowed at each position. 

RosEasy will also ask you for the path to your python executable; make sure you give it the path to the python for your virtual environment. You can always find this path by typing `which python` from a terminal where the `proteindesign` environment is active. 
(You can skip this step in the future by adding an environmental variable - to do so, add the line `ROSEASY_PYTHON=/path/to/anaconda/envs/proteindesign/bin/python` to your `~/.bashrc` file.)

To set up the workspace, enter the following commands:

```bash
roseasy setup design_exercise
```  

Provide RosEasy with the paths to the input PDB file and the resfile. These files should be in protein_design_course_files/design_exercise.

```bash
<input: Python path> /path/to/anaconda/envs/proteindesign/bin/python
<input: PDB file> protein_design_course_files/design_exercise/1LMB_relaxed.pdb
<input: resfile> protein_design_course_files/design_exercise/resfile
```

Once your workspace is set up, `cd` into its root directory.
```bash
cd design_exercise
```

### Interacting with PyRosetta

We'll do the first part of the exercise in a Jupyter notebook. 

Install Jupyter with `conda install jupyter`.

You can download the notebook file used to create this page and follow along. To do so, download the notebook to your workspace's root directory (`design_exercise`; you should already be in this folder) and run Jupyter notebook.

```bash
curl -O https://raw.githubusercontent.com/Kortemme-Lab/protein_design_course_2021/main/course_website/01_RosettaDesign.ipynb
jupyter notebook
``` 
In the browser window that opens, click on `01_RosettaDesign.ipynb`.

In the Jupyter notebook, start with some PyRosetta setup. Before you run any other PyRosetta commands, you must run an init() command. It takes a string of options as an argument. The options we will be using are:  
- `-ex1` - Turns on one level of extra rotamers for more precise packing.  
- `-ex2` - Turns on a second level of extra rotamers.  
- `-use_input_sc` - Adds the rotamers from the input structure to the rotamer set.  

Copy the following into a notebook cell if necessary, and execute it with \<shift\> + \<enter\>.

In [3]:
from pyrosetta import rosetta, init, pose_from_file, create_score_function
from roseasy import pipeline
init('-ex1 -ex2 -use_input_sc')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.11+release.e9f47978df05d6d2de96236c2d949ecf9436018a 2021-03-19T09:06:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r276 2021.11+release.e9f47978df0 e9f47978df05d6d2de96236c2d949ecf9436018a http://www.pyrosetta.org 2021-03-19T09:06:15
core.init: {0} command: PyRosetta -ex1 -ex2 -use_input_sc -database /Users/codykrivacic/software/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-331796401 seed_offset=0 real_seed=-331796401 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed

Next we will **import a pose** and create a workspace object. A pose contains all conformational and sequence information of a protein object. A workspace is a RosEasy class that will help us keep track of parameter files.

In [4]:
pose = pose_from_file('input.pdb.gz')
workspace = pipeline.workspace_from_dir('.')

core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: {0} Total time to initialize 0.953043 seconds.
core.import_pose.import_pose: {0} File 'input.pdb.gz' automatically determined to be of type PDB


### Setting up the PyMOL Observer

For now, it will be instructive to view the models as we make changes. The PyMOL Observer tool in Rosetta automatically sends a new structure to PyMOL whenever a change is made to a pose's coordinates. Before we do that, however, we need to set PyMOL up so it can receive structures from Rosetta.  

Open a new terminal, activate your environment, and open PyMOL:  
```bash
conda activate proteindesign
pymol
```  

In PyMOL, change your directory to the PyRosetta directory, and then run the PyMOLRosettaServer.py script.
```bash
cd /path/to/anaconda/envs/proteindesign/lib/python3.7/site-packages/pyrosetta
run PyMOLRosettaServer.py
```  

PyMOL should display a message that says  
`PyMOL <---> PyRosetta link started!`  

Now we are ready to set up our PyMOL observer. In the Jupyter notebook, enter the following:  

In [5]:
# Setup the PyMOL mover
pymover = rosetta.protocols.moves.AddPyMOLObserver(pose, False)

Now score the input structure: create a scorefunction using the "ref2015" parameters and apply it to the pose.  

In [6]:
sfxn = create_score_function('ref2015')
sfxn(pose)
# Your protein should now appear in the PyMOL window

core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/rama/fd/all.ramaProb
b

-651.2056059515234

The Rosetta score of the pose is -651.2 REU.  

### Fixed-backbone design

Next, let's set up a task factory and movemap for our design run. The TaskFactory will read the resfile we give it, telling Rosetta which positions are allowed to mutate (and to which residue types), which positions are allowed to repack but not mutate, and which positions are not allowed to move at all during packing operations.  

The MoveMap tells certain Rosetta protocols which positions are allowed to move their backbone and sidechain degrees of freedom. In this case, the minimization step of the FastDesign protocol will use the MoveMap to determine which degrees of freedom are to be minimized.  

In [7]:
from pyrosetta.rosetta.core.pack.task.operation import ReadResfile
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.kinematics import MoveMap

# Set up a task factory, which tells Rosetta which residues are designable and/or repackable.
# Our task factory will read the resfile to figure this out, but it is possible to 
# determine designable vs. repackable vs. neither residues using residue selector logic.
taskfactory = TaskFactory()
readresfile = ReadResfile(workspace.resfile_path)
taskfactory.push_back(readresfile)

# Now we set up a movemap. This tells Rosetta which residues are allowed to move their backbone and 
# sidechain degrees of freedom during any minimization steps.
movemap = MoveMap()
# We will be starting out with fixed-backbone design, so let's turn off all backbone degrees of freedom 
# while keeping sidechain DOF on.
movemap.set_bb(False)
movemap.set_chi(True)

Now we are ready to set up a FastDesign mover. FastDesign normally iterates between repacking/designing sidechains and minimizing sidechain + backbone degrees of freedom. Since we turned off backbone degrees of freedom in the MoveMap, it will only minimize sidechain degrees of freedom.   

In [8]:
from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign

# Set up a FastDesign object
fd = FastDesign(sfxn, 3)
fd.set_task_factory(taskfactory)
fd.set_movemap(movemap)
fd.set_scorefxn(sfxn)

protocols.relax.RelaxScriptManager: {0} Reading relax scripts list from database.
protocols.relax.RelaxScriptManager: {0} Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: {0} ================== Reading script file: /Users/codykrivacic/software/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: {0} repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: {0} min 0.01
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: 

In PyMOL, let's color the residues that we will be designing.  
Make a selection by entering the following command in PyMOL's command line:  
`sele designable, (resi 13 or resi 31 or resi 35 or resi 42 or resi 46 or resi 52 or resi 60) and chain A`  
Show them in stick form:  
`show sticks, designable`  
Hide hydrogens:  
`hide everything, hydro`  
Color the selected carbon atoms orange:  
`color orange, designable and name c*`  

Now we are ready to apply the mover and watch what happens.  

In [9]:
# Apply the mover and observe in PyMOL
fd.apply(pose)

protocols.denovo_design.movers.FastDesign: {0} #Packer_Task
protocols.denovo_design.movers.FastDesign: {0} 
protocols.denovo_design.movers.FastDesign: {0} Threads to request: ALL AVAILABLE
protocols.denovo_design.movers.FastDesign: {0} 
protocols.denovo_design.movers.FastDesign: {0} resid	pack?	design?	allowed_aas
protocols.denovo_design.movers.FastDesign: {0} 1	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 2	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 3	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 4	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 5	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 6	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 7	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 8	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 9	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 10	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 11	FALSE	FALSE
protocols.d

protocols.denovo_design.movers.FastDesign: {0} 105	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 106	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 107	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 108	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 109	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 110	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 111	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 112	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 113	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 114	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 115	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 116	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 117	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 118	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 119	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 120	FALS

protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and ch

protocols.relax.FastRelax: {0} CMD: repack  -727.341  0  0  0.30745
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -723.088  0  0  0.31955
protocols.relax.FastRelax: {0} CMD: min  -723.091  0  0  0.31955
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -723.091  0  0  0.31955
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -642.078  0  0  0.55
core.pack.pack_rotamers: {0} built 1129 rotamers at 7 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -642.078  0  0  0.55
protocols.relax.FastRelax: {0} CMD: min  -642.084  0  0  0.55
protocols.relax.FastRelax: {0} MRP: 1  -642.084  -644.011  0  0
protocols.relax.FastRelax: {0} CMD: accept_to_best  -642.084  0  0  0

**How does the packing look, by eye, after your first round of fixed-backbone design? You can manually open the input file in PyMOL (input.pdb.gz) to compare. Does the design look better or worse than the input structure? Was there much change at all? What types of residues do you see being placed in this protein's core?** 

Here is an image of an example output, where orange residues were designed and blue residues are wildtype:

![Example fixbb run](images/ex_fixbb.png)

In this example run, we see that the only mutation that Rosetta produced was L13S. This likely reduces packing interactions, and introduces a possible buried unsatisfied hydrogen bond. Let's see if the score improved.

In [11]:
sfxn(pose)

-644.0114953294067

As we can see, at least in this example, the score actually got worse. Due to the stochasticity of the Monte-Carlo process, Rosetta protocols do not always result in an improved structure, even based on Rosetta's own metrics.  

### Flexible-backbone design

Now we'll run another round of design, this time using a flexible-backbone protocol. Re-import the pose, add it to the PyMOL observer, and edit the MoveMap so that backbone degrees of freedom are enabled for the first 65 residues:  

In [12]:
# Import the pose
pose = pose_from_file('input.pdb.gz')
# Set up the PyMOL observer
pymover = rosetta.protocols.moves.AddPyMOLObserver(pose, False)
# Enable backbone degrees of freedom for the first 65 residues
for i in range(1, 66):
    movemap.set_bb(i, True)
# Add the movemap to the design protocol
fd.set_movemap(movemap)

core.import_pose.import_pose: {0} File 'input.pdb.gz' automatically determined to be of type PDB


Now apply the mover once again and observe in PyMOL.

In [13]:
fd.apply(pose)

protocols.denovo_design.movers.FastDesign: {0} #Packer_Task
protocols.denovo_design.movers.FastDesign: {0} 
protocols.denovo_design.movers.FastDesign: {0} Threads to request: ALL AVAILABLE
protocols.denovo_design.movers.FastDesign: {0} 
protocols.denovo_design.movers.FastDesign: {0} resid	pack?	design?	allowed_aas
protocols.denovo_design.movers.FastDesign: {0} 1	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 2	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 3	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 4	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 5	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 6	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 7	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 8	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 9	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 10	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 11	FALSE	FALSE
protocols.d

protocols.denovo_design.movers.FastDesign: {0} 105	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 106	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 107	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 108	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 109	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 110	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 111	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 112	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 113	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 114	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 115	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 116	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 117	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 118	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 119	FALSE	FALSE
protocols.denovo_design.movers.FastDesign: {0} 120	FALS

protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and chi move
protocols.relax: {0} turning off DNA bb and ch

core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -726.27  0.266268  0.266268  0.30745
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -720.599  0.266268  0.266268  0.31955
protocols.relax.FastRelax: {0} CMD: min  -726.56  0.205641  0.205641  0.31955
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -726.56  0.205641  0.205641  0.31955
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -633.067  0.205641  0.205641  0.55
core.pack.pack_rotamers: {0} built 980 rotamers at 7 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating PDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available thre

You may notice that, even though we only allowed the first 65 residues to move their backbone degrees of freedom, the entire chain moved relative to the rest of the structure. This is what's referred to as the "lever arm effect", and will be discussed in more detail when we talk about loop modeling.  

Let's score the pose again.

In [14]:
sfxn(pose)

-646.7160039909187

The design scores slightly better in this case. Note that even with backbone degrees of freedom, design will not always result in a score that is lower than the input structure. Hence, most design projects involve creating thousands of designs and then filtering them down to the most promising.  

We can see from these results that Rosetta found a different possible solution to the packing of this protein's internal core. Once again, designed residues are in orange, input in blue.  
![fastdesign_ex](images/fastdesign_ex.png)


### (Optional) Editing the resfile

One thing that can further help optimize designs it to allow positions that are adjacent to the designed positions to repack. Examine the input structure in PyMOL and select a few (no more than 5 for the sake of computational expense) residues that neighbor the design positions. Then, open the resfile (`project_params/resfile`) in a text editor and add set these positions to be repackable by adding lines with the following format:

```
RESNUM CHAIN NATAA
```

So, for example, if I want position 18 to be repackable, I would add the line:

```
18 A NATAA
```

**Which positions did you pick for repacking? Why did you pick them?**

### Creating multiple designs

Now let's create a few structures, using three different protocols. The three protocols will do the following:  
- Fixed-backbone design: Runs design without backbone degrees of freedom.
- Fastdesign: Runs design with backbone degrees of freedom. To limit the amount of computational resources required, we will only allow backbone degrees of freedom for designed or repacked residues.
- Layerdesign: Runs flexible backbone design while limiting allowable residues, depending on how solvent-accessible those positions are.  

Cancel out of the Jupyter session and then type one of the following commands from your workspace's root directory. Have each person in your group run a different command; if necessary, double up on the layerdesign command.

```bash
roseasy submit . standard_params/fixbb.py -l -n 10
roseasy submit . standard_params/fastdesign.py -l -n 5
roseasy submit . standard_params/layerdesign.py -l -n 5
```

This will cause Rosetta to run 5-10 simulations back-to-back, either using fixed-backbone design (`fixbb.py`), flexible-backbone design (`fastdesign.py`), or flexible-backbone design with layer design (`layerdesign.py`). If time permits (the above commands should take around 10-15 mintues), You can increase the number of designs Rosetta produces by changing the number at the end of the command. Output `.pdb.gz` files will be dumped into the `01_designs/outputs/` folder. If you run multiple `submit` commands, each command will create a new design folder, incrementing the number at the start.  

### Plotting design structures

Once your job is complete (should take ~5 - 20 minutes), open a RosEasy plotting graphical user interface:
```bash
roseasy plot 01_designs/outputs/
```

The GUI allows you to plot any metric, including the Rosetta score, against various other metrics that are calculatd by the script. To change the X- and Y-axis, select a different metric from the drop-down menus below the scatter plot (you may need to resize the window to see the drop-down menus). We will discuss these metrics in a later class. For now, plot the "packstat" score against the Rosetta total score. The PackStat score probes protein cores to find cavities; a higher score indicates fewer cavities and is therefore better. **How does the PackStat score compare between the different methods? What accounts for this trend?**

Examine a your designs by right-clicking on each point in the plot, then selecting "compare to input". If you have the `wt_vs_mut` PyMOL plugin installed, you can type "ww" in PyMOL to compare individual mutations. The wildtype residue will appear in white, mutant in yellow. 

**Pick a few interesting mutations and describe what is happening. Is an existing packing interaction being displaced? Is Rosetta forming an internal salt bridge? Do you see a buried unsatisfied hydrogen bond?**

**Compare your group's flexible-backbone results with your group's layerdesign results. Visually compare average runtimes by plotting run time on the X-axis. Was there a significant reduction in run time for layered design vs. flexible backbone design? (Computer speed and temperature may confound this, but see if you observe a general trend.) How do the scores compare between the two design methods?**

### Analyzing design sequences

Now that you've looked at your designs, let's summarize them in a table so we can look at their sequence similarity to the experimental results. Run:  
`roseasy make_table 01_designs/`

Roseasy will create an Excel file that summarizes a bunch of design parameters. The important column is 'Resfile sequence', which contains the residue identities of all the designable positions. The ordering is the same as in Figure 2. 

**With your group, compare your outputs to those from the Lim paper. Looking at figure 2, how many of your designs are "functional"? Partially functional? Non-functional? Wildtype? Most will likely have no data, but based on your group's sequences, which, if any, design method best recapitulated the experimental results?**


### Rosetta evaluation of known sequences

Finally, pick a functional sequence from the Lim paper, preferably one with a relatively high estimated core van der Waals volume, and edit the resfile so that Rosetta is forced to recapitulate that sequence. To do this, edit each line so that instead of 

`## A ALLAA`

it says

`## A PIKAA RES`.

Note that the residue numbers in the resfile are different than those in the paper, but the ordering is the same (each resfile residue number is 5 less than the number of the corresponding residue in the paper). For example, if I want position 18 (using the paper's numbering) to be a valine, I would change the line `13 A ALLAA` to `13 A PIKAA V`. Make sure you do not rename the resfile. 

Run a few flexible backbone simulations using the new resfile (if you have time, increase the number of simulations):

`roseasy submit . standard_params/fastdesign.py -l -n 3`

This design will be placed in a new folder, `02_designs/outputs/`.

**How do these designs score compared to the other designs found by flexible backbone and layered design in your group? Why did we use the flexible backbone procedure here, even though we know the sequence is functional? What do the demonstrate about scoring and sampling in Rosetta?**